## Run Ollama in Colab
[List of available Ollama LLMs.](https://ollama.com/library)  

#### Here's a list of isses to take care of:
1. Insert a Colab badge.  
1. Mention here that it only works on Colab so far, not Windows.  
1. Break this notebook down to separate .py files to be sourced.  
1. **Managing Ollama Server Lifecycles:**
    Currently, you use a background process (ollama serve). Consider a controlled lifecycle using Docker containers or managed processes (e.g., via supervisord or systemd).   
1. [x] Apply "**Explicit Error Handling**" for each of the shell commands (see chat)  
1. [ ] **Resource Monitoring & Logging:**  
    Capture and monitor resource utilization (CPU/GPU, memory usage) to ensure sustainable performance.  

In [1]:
# This is a .py script to be sourced: (I will need to format it as a separate file to be called with a variable llm_name)
import shutil
import subprocess
from time import sleep
import requests

# Choice of LLM:
llm_name = "mistral-small"  # "mistral"

# Install Ollama via shell
if shutil.which('ollama') is None:
    print("Ollama not found, installing...")
    shell_output_curl_command = subprocess.run(
        'curl https://ollama.ai/install.sh | sh',
        capture_output=True, text=True, shell=True
    )
    if shell_output_curl_command.returncode != 0:
        raise RuntimeError(f"Error installing Ollama: {shell_output_curl_command.stderr}")
else:
    print("Ollama is already installed.")

# Start Ollama server in background
print("Starting Ollama server...")
process_serve = subprocess.Popen(
    'OLLAMA_HOST=127.0.0.1:11434 ollama serve > serve.log 2>&1 &',
    stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True
)
print(f"Started Ollama process with PID: {process_serve.pid}")

# Function to ensure Ollama is ready
def wait_for_ollama_ready(timeout=15):
    print("Waiting for Ollama server to be ready...")
    for _ in range(timeout):
        try:
            response = requests.get("http://localhost:11434")
            if response.status_code == 200:
                print("Ollama server is ready.")
                return
        except requests.exceptions.ConnectionError:
            sleep(1)
    raise RuntimeError("Ollama server failed to start within timeout.")

wait_for_ollama_ready()

# Pull LLM model
print(f"Pulling '{llm_name}' LLM model...")
shell_output_pull_LLM = subprocess.run(
    f'ollama pull {llm_name}', capture_output=True, text=True, shell=True
)
if shell_output_pull_LLM.returncode != 0:
    raise RuntimeError(f"Error pulling '{llm_name}': {shell_output_pull_LLM.stderr}")

# Verify available models
shell_output_models_list = subprocess.run(
    'ollama list', capture_output=True, text=True, shell=True
)
if shell_output_models_list.returncode != 0:
    raise RuntimeError(f"Error listing models: {shell_output_models_list.stderr}")
else:
    print(f"Available models:\n{shell_output_models_list.stdout}")

# Install LangChain Ollama integration
print("Installing langchain-ollama via pip.")
pip_langchainollama_command = subprocess.run(
    'pip install -U langchain-ollama', capture_output=True, text=True, shell=True
)
if pip_langchainollama_command.returncode != 0:
    raise RuntimeError(f"Error installing 'langchain-ollama': {pip_langchainollama_command.stderr}")

# Import and configure LLM
from langchain_ollama.llms import OllamaLLM
model = OllamaLLM(model=llm_name)

print("LLM setup complete and ready for use.")


Ollama not found, installing...
Starting Ollama server...
Started Ollama process with PID: 3287
Waiting for Ollama server to be ready...
Ollama server is ready.
Pulling 'mistral-small' LLM model...
Available models:
NAME                    ID              SIZE     MODIFIED               
mistral-small:latest    8039dd90c113    14 GB    Less than a second ago    

Installing langchain-ollama via pip.
LLM setup complete and ready for use.


In [2]:
from langchain_core.prompts import ChatPromptTemplate

template = """Question: {question}

Answer: Provide concise and simple answer!"""

prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model

print(chain.invoke({"question": "What is a good way to continue this sentence: 'you are a ...'? It has to by syntactically correct!"}))

A good way to continue that sentence could be:

"You are a friend."

Or

"You are a teacher."
